In [1]:
import pickle
import numpy as np
import pandas as pd
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv
from sklearn.metrics import accuracy_score

In [2]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

In [3]:
with open('basic_pose.pkl', 'rb') as f:
    model = pickle.load(f)

In [4]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [5]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [ ]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)

        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        
        # Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        # Displaying the output
        
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            row = pose_row
                            
            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0] #예측힌 클래스들 저장
            body_language_prob = model.predict_proba(X)[0] #확률 저장
            
            print(body_language_class, body_language_prob)
        
            
            # display action near the ear
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            
            #Calculate and Display angle
            
            # Get right side coordinates
            right_shoulder = [pose[mp_holistic.PoseLandmark.RIGHT_SHOULDER.value].x,pose[mp_holistic.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_hip = [pose[mp_holistic.PoseLandmark.RIGHT_HIP.value].x,pose[mp_holistic.PoseLandmark.RIGHT_HIP.value].y]
            right_knee = [pose[mp_holistic.PoseLandmark.RIGHT_KNEE.value].x,pose[mp_holistic.PoseLandmark.RIGHT_WRIST.value].y]
            
            # Calculate angle
            right_angle = calculate_angle(left_shoulder, left_hip, left_knee)
            
            # Visualize angle
            cv2.putText(image, str(right_angle), 
                           tuple(np.multiply(right_hip, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA           
                       )
                        
                        
                        
            # Get left side coordinates
            left_shoulder = [pose[mp_holistic.PoseLandmark.LEFT_SHOULDER.value].x,pose[mp_holistic.PoseLandmark.LEFT_SHOULDER.value].y]
            left_hip = [pose[mp_holistic.PoseLandmark.LEFT_HIP.value].x,pose[mp_holistic.PoseLandmark.LEFT_HIP.value].y]
            left_knee = [pose[mp_holistic.PoseLandmark.LEFT_KNEE.value].x,pose[mp_holistic.PoseLandmark.LEFT_WRIST.value].y]
            
            # Calculate angle
            left_angle = calculate_angle(left_shoulder, left_hip, left_knee)
            
            # Visualize angle
            cv2.putText(image, str(left_angle), 
                           tuple(np.multiply(left_hip, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (200, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        except:
            pass

                        
        cv2.imshow('Raw Webcam Feed', cv2.flip(image, 1))
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

sit [0.86 0.14]
sit [0.86 0.14]
sit [0.86 0.14]
sit [0.86 0.14]
sit [0.86 0.14]
sit [0.86 0.14]
sit [0.86 0.14]
sit [0.86 0.14]
sit [0.86 0.14]
sit [0.89 0.11]
sit [0.92 0.08]
sit [0.82 0.18]
sit [0.84 0.16]
sit [0.9 0.1]
sit [0.96 0.04]
sit [0.97 0.03]
sit [0.97 0.03]
sit [0.97 0.03]
sit [0.97 0.03]
sit [0.97 0.03]
sit [0.97 0.03]
sit [0.97 0.03]
sit [0.93 0.07]
sit [0.86 0.14]
sit [0.82 0.18]
sit [0.72 0.28]
sit [0.8 0.2]
sit [0.76 0.24]
sit [0.77 0.23]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.79 0.21]
sit [0.78 0.22]
sit [0.79 0.21]
sit [0.85 0.15]
sit [0.89 0.11]
sit [0.89 0.11]
sit [0.85 0.15]
sit [0.92 0.08]
sit [0.86 0.14]
sit [0.83 0.17]
sit [0.83 0.17]
sit [0.83 0.17]
sit [0.84 0.16]
sit [0.82 0.18]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8

sit [0.84 0.16]
sit [0.84 0.16]
sit [0.84 0.16]
sit [0.86 0.14]
sit [0.86 0.14]
sit [0.88 0.12]
sit [0.88 0.12]
sit [0.83 0.17]
sit [0.83 0.17]
sit [0.83 0.17]
sit [0.83 0.17]
sit [0.83 0.17]
sit [0.8 0.2]
sit [0.82 0.18]
sit [0.83 0.17]
sit [0.83 0.17]
sit [0.82 0.18]
sit [0.78 0.22]
sit [0.78 0.22]
sit [0.78 0.22]
sit [0.79 0.21]
sit [0.83 0.17]
sit [0.83 0.17]
sit [0.8 0.2]
sit [0.83 0.17]
sit [0.8 0.2]
sit [0.83 0.17]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.77 0.23]
sit [0.76 0.24]
sit [0.76 0.24]
sit [0.73 0.27]
sit [0.74 0.26]
sit [0.72 0.28]
sit [0.72 0.28]
sit [0.72 0.28]
sit [0.72 0.28]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.72 0.28]
sit [0.73 0.27]
sit [0.73 0.27]
sit [0.72 0.28]
sit [0.72 0.28]
sit [0.72 0.28]
sit [0.72 0.28]
sit [0.72 0.28]
sit [0.73 0.27]
sit [0.73 0.27]
sit [0.73 0.27]
sit [0.78 0.22]
sit [0.79 0.21]
sit [0.79 0.21]
sit [0.79 0.21]
sit [0.76 0.24]
sit [0.76 0.24]
sit [0.74 0.26]
sit [0.74 0.26]
sit [0.74 0.26]
sit [0.74 0.26]
sit [0.74 0.26]
sit 

sit [0.75 0.25]
sit [0.73 0.27]
sit [0.72 0.28]
sit [0.72 0.28]
sit [0.73 0.27]
sit [0.75 0.25]
sit [0.8 0.2]
sit [0.75 0.25]
sit [0.72 0.28]
sit [0.83 0.17]
sit [0.81 0.19]
sit [0.83 0.17]
sit [0.83 0.17]
sit [0.89 0.11]
sit [0.74 0.26]
sit [0.73 0.27]
sit [0.75 0.25]
sit [0.74 0.26]
sit [0.73 0.27]
sit [0.78 0.22]
sit [0.77 0.23]
sit [0.83 0.17]
sit [0.83 0.17]
sit [0.79 0.21]
sit [0.87 0.13]
sit [0.87 0.13]
sit [0.85 0.15]
sit [0.72 0.28]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.83 0.17]
sit [0.86 0.14]
sit [0.89 0.11]
sit [0.93 0.07]
sit [0.92 0.08]
sit [0.92 0.08]
sit [0.94 0.06]
sit [0.94 0.06]
sit [0.94 0.06]
sit [0.94 0.06]
sit [0.94 0.06]
sit [0.94 0.06]
sit [0.94 0.06]
sit [0.94 0.06]
sit [0.94 0.06]
sit [0.94 0.06]
sit [0.92 0.08]
sit [0.91 0.09]
sit [0.87 0.13]
sit [0.87 0.13]
sit [0.87 0.13]
sit [0.94 0.06]
sit [0.91 0.09]
sit [0.91 0.09]
sit [0.87 0.13]
sit [0.81 0.19]
sit [0.83 0.17]
sit [0.83 0.17]
sit [0.8 0.2]
sit [0.83 0.17]
sit [0.84 0.16]
sit [0.86 0.14]
sit [0.86 0.

sit [0.72 0.28]
sit [0.72 0.28]
sit [0.72 0.28]
sit [0.76 0.24]
sit [0.76 0.24]
sit [0.76 0.24]
sit [0.76 0.24]
sit [0.76 0.24]
sit [0.76 0.24]
sit [0.73 0.27]
sit [0.76 0.24]
sit [0.76 0.24]
sit [0.73 0.27]
sit [0.73 0.27]
sit [0.73 0.27]
sit [0.73 0.27]
sit [0.73 0.27]
sit [0.73 0.27]
sit [0.73 0.27]
sit [0.73 0.27]
sit [0.73 0.27]
sit [0.73 0.27]
sit [0.73 0.27]
sit [0.73 0.27]
sit [0.72 0.28]
sit [0.72 0.28]
sit [0.72 0.28]
sit [0.72 0.28]
sit [0.72 0.28]
sit [0.72 0.28]
sit [0.72 0.28]
sit [0.72 0.28]
sit [0.72 0.28]
sit [0.76 0.24]
sit [0.73 0.27]
sit [0.73 0.27]
sit [0.74 0.26]
sit [0.76 0.24]
sit [0.76 0.24]
sit [0.76 0.24]
sit [0.76 0.24]
sit [0.73 0.27]
sit [0.76 0.24]
sit [0.77 0.23]
sit [0.77 0.23]
sit [0.76 0.24]
sit [0.76 0.24]
sit [0.73 0.27]
sit [0.74 0.26]
sit [0.74 0.26]
sit [0.73 0.27]
sit [0.74 0.26]
sit [0.76 0.24]
sit [0.76 0.24]
sit [0.76 0.24]
sit [0.63 0.37]
sit [0.54 0.46]
sit [0.61 0.39]
sit [0.5 0.5]
sit [0.63 0.37]
sit [0.76 0.24]
sit [0.65 0.35]
sit [0.68 

sit [0.79 0.21]
sit [0.81 0.19]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.79 0.21]
sit [0.75 0.25]
sit [0.74 0.26]
sit [0.74 0.26]
sit [0.74 0.26]
sit [0.76 0.24]
sit [0.8 0.2]
sit [0.78 0.22]
sit [0.78 0.22]
sit [0.78 0.22]
sit [0.77 0.23]
sit [0.77 0.23]
sit [0.77 0.23]
sit [0.77 0.23]
sit [0.77 0.23]
sit [0.77 0.23]
sit [0.77 0.23]
sit [0.77 0.23]
sit [0.79 0.21]
sit [0.83 0.17]
sit [0.79 0.21]
sit [0.75 0.25]
sit [0.75 0.25]
sit [0.75 0.25]
sit [0.58 0.42]
sit [0.58 0.42]
sit [0.58 0.42]
sit [0.58 0.42]
sit [0.58 0.42]
sit [0.58 0.42]
sit [0.63 0.37]
sit [0.63 0.37]
sit [0.75 0.25]
sit [0.63 0.37]
sit [0.58 0.42]
sit [0.62 0.38]
sit [0.75 0.25]
sit [0.75 0.25]
sit [0.8 0.2]
sit [0.85 0.15]
sit [0.88 0.12]
sit [0.74 0.26]
sit [0.73 0.27]
sit [0.68 0.32]
sit [0.79 0.21]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.62 0.38]
sit [0.63 0.37]
sit [0.81 0.19]
sit [0.86 0.14]
sit [0.86 0.14]
sit [0.86 0.14]
sit [0.81 0.19]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.78 0.22]
sit [0.77 0.

...

sit [0.71 0.29]
sit [0.71 0.29]
sit [0.7 0.3]
sit [0.7 0.3]
sit [0.7 0.3]
sit [0.7 0.3]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.7 0.3]
sit [0.7 0.3]
sit [0.71 0.29]
sit [0.72 0.28]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.72 0.28]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.72 0.28]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.7 0.3]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.7 0.3]
sit [0.71 0.29]
sit [0.72 0.28]
sit [0.7 0.3]
sit [0.72 0.28]
sit [0.72 0.28]
sit [0.73 0.27]
sit [0.73 0.27]
sit [0.73 0.27]
sit [0.73 0.27]
sit [0.73 0.27]
sit [0.73 0.27]
sit [0.73 0.27]
sit [0.73 0.27]
sit [0.73 0.27]
sit [0.72 0.28]
sit [0.72 0.28]
sit [0.72 0.28]
sit [0.72 0.28]
sit [0.71 0.29]
sit [0.72 0.28]
sit [0.72 0.28]
sit [0.73 0.27]
sit [0.72 0.28]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.7 0.3]
sit [0.71 0.29]
sit [0.7 0.3]
sit [0.7 0.3]
sit [0.7 0.3]
si

sit [0.79 0.21]
sit [0.79 0.21]
sit [0.79 0.21]
sit [0.79 0.21]
sit [0.79 0.21]
sit [0.79 0.21]
sit [0.79 0.21]
sit [0.79 0.21]
sit [0.79 0.21]
sit [0.79 0.21]
sit [0.79 0.21]
sit [0.79 0.21]
sit [0.79 0.21]
sit [0.79 0.21]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.79 0.21]
sit [0.79 0.21]
sit [0.78 0.22]
sit [0.78 0.22]
sit [0.78 0.22]
sit [0.78 0.22]
sit [0.78 0.22]
sit [0.78 0.22]
sit [0.78 0.22]
sit [0.77 0.23]
sit [0.77 0.23]
sit [0.76 0.24]
sit [0.77 0.23]
sit [0.74 0.26]
sit [0.72 0.28]
sit [0.73 0.27]
sit [0.72 0.28]
sit [0.71 0.29]
sit [0.73 0.27]
sit [0.73 0.27]
sit [0.73 0.27]
sit [0.73 0.27]
sit [0.73 0.27]
sit [0.73 0.27]
sit [0.76 0.24]
sit [0.76 0.24]
sit [0.76 0.24]
sit [0.76 0.24]
sit [0.76 0.24]
sit [0.76 0.24]
sit [0.76 0.24]
sit [0.76 0.24]
sit [0.76 0.24]
sit [0.76 0.24]
sit [0.76 0.24]
sit [0.76 0.24]
sit [0.75 0.25]
sit [0.75 0.25]
sit [0.75 0.25]
sit [0.75 0.25]
sit [0.76 0.24]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.75 0.25]
sit [0.75 0.

sit [0.88 0.12]
sit [0.91 0.09]
sit [0.91 0.09]
sit [0.91 0.09]
sit [0.91 0.09]
sit [0.91 0.09]
sit [0.91 0.09]
sit [0.91 0.09]
sit [0.92 0.08]
sit [0.92 0.08]
sit [0.91 0.09]
sit [0.91 0.09]
sit [0.91 0.09]
sit [0.91 0.09]
sit [0.91 0.09]
sit [0.91 0.09]
sit [0.91 0.09]
sit [0.91 0.09]
sit [0.87 0.13]
sit [0.84 0.16]
sit [0.84 0.16]
sit [0.84 0.16]
sit [0.84 0.16]
sit [0.84 0.16]
sit [0.84 0.16]
sit [0.84 0.16]
sit [0.84 0.16]
sit [0.84 0.16]
sit [0.84 0.16]
sit [0.84 0.16]
sit [0.83 0.17]
sit [0.83 0.17]
sit [0.82 0.18]
sit [0.81 0.19]
sit [0.79 0.21]
sit [0.79 0.21]
sit [0.79 0.21]
sit [0.79 0.21]
sit [0.79 0.21]
sit [0.79 0.21]
sit [0.8 0.2]
sit [0.83 0.17]
sit [0.8 0.2]
sit [0.86 0.14]
sit [0.88 0.12]
sit [0.74 0.26]
sit [0.9 0.1]
sit [0.88 0.12]
sit [0.88 0.12]
sit [0.88 0.12]
sit [0.86 0.14]
sit [0.86 0.14]
sit [0.86 0.14]
sit [0.86 0.14]
sit [0.86 0.14]
sit [0.86 0.14]
sit [0.86 0.14]
sit [0.86 0.14]
sit [0.86 0.14]
sit [0.86 0.14]
sit [0.85 0.15]
sit [0.85 0.15]
sit [0.85 0.15

sit [0.82 0.18]
sit [0.79 0.21]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.79 0.21]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.79 0.21]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.79 0.21]
sit [0.79 0.21]
sit [0.79 0.21]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.79 0.21]
sit [0.79 0.21]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.8 0.2]
sit [0.86 0.14]
sit [0.86 0.14]
sit [0.85 0.15]
sit [0.86 0.14]
sit [0.84 0.16]
sit [0.84 0.16]
sit [0.85 0.15]
sit [0.83 0.17]
sit [0.81 0.19]
sit [0.81 0.19]
sit [0.81 0.19]
sit [0.81 0.19]
sit [0.8 0.2]
sit [0.68 0.32]
sit [0.67 0.33]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.67 0.33]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.73 0.27]
sit [0.72 0.28]
sit [0.74 0.26]
sit [0.74 0.26]
sit [0.82 0.18]
sit [0.8 0.2]
sit [0.79 0.21]
sit [0.8 0.2]
sit [0.8 0.2]
si

sit [0.7 0.3]
sit [0.7 0.3]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.74 0.26]
sit [0.74 0.26]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.71 0.29]
sit [0.72 0.28]
sit [0.72 0.28]
sit [0.72 0.28]
sit [0.73 0.27]
sit [0.74 0.26]
sit [0.74 0.26]
sit [0.67 0.33]
sit [0.69 0.31]
sit [0.59 0.41]
sit [0.56 0.44]
sit [0.56 0.44]
sit [0.72 0.28]
sit [0.72 0.28]
sit [0.72 0.28]
sit [0.74 0.26]
sit [0.76 0.24]
sit [0.79 0.21]
sit [0.8 0.2]
sit [0.82 0.18]
sit [0.82 0.18]
sit [0.82 0.18]
sit [0.74 0.26]
sit [0.65 0.35]
sit [0.65 0.35]
sit [0.68 0.32]
sit [0.67 0.33]
sit [0.69 0.31]
sit [0.72 0.28]
sit [0.69 0.31]
sit [0.69 0.31]
sit [0.81 0.19]
sit [0.78 0.22]
sit [0.8 0.2]
sit [0.78 0.22]
sit [0.8 0.2]
si